In [ ]:
#supress warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import sys
import os
_stderr = sys.stderr
null = open(os.devnull,'wb')

import scanpy as sc
#set some figure parameters for nice display inside jupyternotebooks.
%matplotlib inline
sc.settings.set_figure_params(dpi=80, frameon=False, figsize=(5, 5), facecolor='white')

## Prep our RNA adata structure

In [ ]:
adata = sc.read_10x_h5("/rugpfs/fs0/tavz_lab/scratch/amillet/11_16_2022_Alon/E4AD_1yr-ATAC/outs/filtered_feature_bc_matrix.h5")
adata.var_names_make_unique()
adata

In [ ]:
import pandas as pd
import numpy as np
cell_annot = pd.read_csv('/ru-auth/local/home/amillet/Multivelo/E4AD_1yr/Mglia_Only/annots.csv', sep=',', index_col=0)

In [ ]:
adata = adata[np.isin(adata.obs.index,cell_annot.index)]

In [ ]:
adata = adata[cell_annot.index,:]
adata.obs['celltype'] = cell_annot['mglia_ident']

In [ ]:
sc.pp.filter_genes(adata, min_cells=3)

In [ ]:
adata.raw = adata
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)
sc.pp.highly_variable_genes(adata, min_mean=0.0125, max_mean=3, min_disp=0.5)
adata = adata[:, adata.var.highly_variable]
sc.pp.scale(adata, max_value=10)

In [ ]:
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=10)
sc.tl.umap(adata)
sc.pl.umap(adata, color = 'celltype')

In [ ]:
adata.write(('E4AD_1yr/rna_adata.h5ad'), compression='gzip')

## Prep our ATAC adata structure

In [ ]:
import os
import pycisTopic
#set some figure parameters for nice display inside jupyternotebooks.
%matplotlib inline

In [ ]:
import scanpy as sc
adata = sc.read_h5ad('E4AD_1yr/rna_adata.h5ad')
cell_data = adata.obs
cell_data['sample_id'] = 'E4AD_1yr'
cell_data['celltype'] = cell_data['celltype'].astype(str) # set data type of the celltype column to str, otherwise the export_pseudobulk function will complain.
del(adata)

In [ ]:
# Get chromosome sizes (for mm10 here)
import pyranges as pr
import requests
import pandas as pd
target_url='https://hgdownload.cse.ucsc.edu/goldenPath/mm10/bigZips/mm10.chrom.sizes'
chromsizes=pd.read_csv(target_url, sep='\t', header=None)
chromsizes.columns=['Chromosome', 'End']
chromsizes['Start']=[0]*chromsizes.shape[0]
chromsizes=chromsizes.loc[:,['Chromosome', 'Start', 'End']]
# Exceptionally in this case, to agree with CellRangerARC annotations
chromsizes['Chromosome'] = [chromsizes['Chromosome'][x].replace('v', '.') for x in range(len(chromsizes['Chromosome']))]
chromsizes['Chromosome'] = [chromsizes['Chromosome'][x].split('_')[1] if len(chromsizes['Chromosome'][x].split('_')) > 1 else chromsizes['Chromosome'][x] for x in range(len(chromsizes['Chromosome']))]
chromsizes=pr.PyRanges(chromsizes)

In [ ]:
# there are some rando chromosomes in the fragments file that are not present in chromsizes. to filter them,
# i write chromsizes to a csv and will use this in R to quickly do filtering of the fragments tsv
chromsizes.to_csv("mm10_chromsizes.csv")

In [ ]:
# processing being done in scenicplus/E4AD_1yr/frag_filtering.R
# the new, filtered fragments file is being saved as scenicplus/E4AD_1yr/atac_fragments_filtered.tsv.gz
fragments_dict = {'E4AD_1yr': '/lustre/fs4/home/amillet/scenicplus/E4AD_1yr/atac_fragments_filtered.tsv.gz'}

In [ ]:
from pycisTopic.pseudobulk_peak_calling import export_pseudobulk
bw_paths, bed_paths = export_pseudobulk(input_data = cell_data,
                 variable = 'celltype',                                                                     # variable by which to generate pseubulk profiles, in this case we want pseudobulks per celltype
                 sample_id_col = 'sample_id',
                 chromsizes = chromsizes,
                 bed_path = '/lustre/fs4/home/amillet/scenicplus/E4AD_1yr/consensus_peak_calling/pseudobulk_bed_files/',  # specify where pseudobulk_bed_files should be stored
                 bigwig_path = '/lustre/fs4/home/amillet/scenicplus/E4AD_1yr/consensus_peak_calling/pseudobulk_bw_files/',# specify where pseudobulk_bw_files should be stored
                 path_to_fragments = fragments_dict,                                                        # location of fragment fiels
                 n_cpu = 8,                                                                                 # specify the number of cores to use, we use ray for multi processing
                 normalize_bigwig = True,
                 remove_duplicates = True,
                 _temp_dir = "/lustre/fs4/home/amillet/ray_spill",      
                 split_pattern = '-')

In [ ]:
import pickle
pickle.dump(bed_paths,
            open('E4AD_1yr/consensus_peak_calling/pseudobulk_bed_files/bed_paths.pkl', 'wb'))
pickle.dump(bw_paths,
           open('E4AD_1yr/consensus_peak_calling/pseudobulk_bed_files/bw_paths.pkl', 'wb'))

In [ ]:
bed_paths

In [ ]:
import pickle
bed_paths = pickle.load(open('E4AD_1yr/consensus_peak_calling/pseudobulk_bed_files/bed_paths.pkl', 'rb'))
bw_paths =  pickle.load(open('E4AD_1yr/consensus_peak_calling/pseudobulk_bed_files/bw_paths.pkl', 'rb'))
from pycisTopic.pseudobulk_peak_calling import peak_calling
macs_path='macs2'
# Run peak calling
narrow_peaks_dict = peak_calling(macs_path,
                                 bed_paths,
                                 'E4AD_1yr/consensus_peak_calling/MACS/',
                                 genome_size='mm',
                                 n_cpu=8,
                                 input_format='BEDPE',
                                 shift=73,
                                 ext_size=146,
                                 keep_dup = 'all',
                                 q_value = 0.05,
                                 _temp_dir = "/lustre/fs4/home/amillet/ray_spill")

In [ ]:
# ray has some weird rules about the temp_dir name length.
# had to choose a folder close to home for actual run,
# transferring to scenicplus/E4AD_1yr folder after the fact.
import shutil
shutil.move("/lustre/fs4/home/amillet/ray_spill", "/lustre/fs4/home/amillet/scenicplus/E4AD_1yr")

In [ ]:
pickle.dump(narrow_peaks_dict,
            open('E4AD_1yr/consensus_peak_calling/MACS/narrow_peaks_dict.pkl', 'wb'))

In [ ]:
from pycisTopic.iterative_peak_calling import *
# Other param
peak_half_width = 250
path_to_blacklist= 'mm10-blacklist.v2.bed' #downloaded from aertslab github
# Get consensus peaks
consensus_peaks=get_consensus_peaks(narrow_peaks_dict, peak_half_width, chromsizes=chromsizes, path_to_blacklist=path_to_blacklist)

In [ ]:
consensus_peaks.to_bed(
    path = 'E4AD_1yr/consensus_peak_calling/consensus_regions.bed',
    keep=True,
    compression='infer',
    chain=False)

### QC

In [ ]:
import pybiomart as pbm
dataset = pbm.Dataset(name='mmusculus_gene_ensembl',  host='http://www.ensembl.org')
annot = dataset.query(attributes=['chromosome_name', 'transcription_start_site', 'strand', 'external_gene_name', 'transcript_biotype'])
annot['Chromosome/scaffold name'] = annot['Chromosome/scaffold name'].to_numpy(dtype = str)
filter = annot['Chromosome/scaffold name'].str.contains('CHR|GL|JH|MT')
annot = annot[~filter]
annot['Chromosome/scaffold name'] = annot['Chromosome/scaffold name'].str.replace(r'(\b\S)', r'chr\1')
annot.columns=['Chromosome', 'Start', 'Strand', 'Gene', 'Transcript_type']
annot = annot[annot.Transcript_type == 'protein_coding']
from pycisTopic.qc import *
path_to_regions = {'E4AD_1yr':'E4AD_1yr/consensus_peak_calling/consensus_regions.bed'}

metadata_bc, profile_data_dict = compute_qc_stats(
                fragments_dict = fragments_dict,
                tss_annotation = annot,
                stats=['barcode_rank_plot', 'duplicate_rate', 'insert_size_distribution', 'profile_tss', 'frip'],
                label_list = None,
                path_to_regions = path_to_regions,
                n_cpu = 1,
                valid_bc = None,
                n_frag = 100,
                n_bc = None,
                tss_flank_window = 1000,
                tss_window = 50,
                tss_minimum_signal_window = 100,
                tss_rolling_window = 10,
                remove_duplicates = True,
                _temp_dir = "/lustre/fs4/home/amillet/ray_spill")

if not os.path.exists('E4AD_1yr/quality_control'):
    os.makedirs('E4AD_1yr/quality_control')

pickle.dump(metadata_bc,
            open('E4AD_1yr/quality_control/metadata_bc.pkl', 'wb'))

pickle.dump(profile_data_dict,
            open('E4AD_1yr/quality_control/profile_data_dict.pkl', 'wb'))

In [ ]:
                        #[min,  #max]
QC_filters = {
    'Log_unique_nr_frag': [3.3 , None],
    'FRIP':               [0.4, None],
    'TSS_enrichment':     [1   , None],
    'Dupl_rate':          [None, None]

}

# Return figure to plot together with other metrics, and cells passing filters. Figure will be saved as pdf.
from pycisTopic.qc import *
FRIP_NR_FRAG_fig, FRIP_NR_FRAG_filter=plot_barcode_metrics(metadata_bc['E4AD_1yr'],
                                       var_x='Log_unique_nr_frag',
                                       var_y='FRIP',
                                       min_x=QC_filters['Log_unique_nr_frag'][0],
                                       max_x=QC_filters['Log_unique_nr_frag'][1],
                                       min_y=QC_filters['FRIP'][0],
                                       max_y=QC_filters['FRIP'][1],
                                       return_cells=True,
                                       return_fig=True,
                                       plot=False)
# Return figure to plot together with other metrics, and cells passing filters
TSS_NR_FRAG_fig, TSS_NR_FRAG_filter=plot_barcode_metrics(metadata_bc['E4AD_1yr'],
                                      var_x='Log_unique_nr_frag',
                                      var_y='TSS_enrichment',
                                      min_x=QC_filters['Log_unique_nr_frag'][0],
                                      max_x=QC_filters['Log_unique_nr_frag'][1],
                                      min_y=QC_filters['TSS_enrichment'][0],
                                      max_y=QC_filters['TSS_enrichment'][1],
                                      return_cells=True,
                                      return_fig=True,
                                      plot=False)
# Return figure to plot together with other metrics, but not returning cells (no filter applied for the duplication rate  per barcode)
DR_NR_FRAG_fig=plot_barcode_metrics(metadata_bc['E4AD_1yr'],
                                      var_x='Log_unique_nr_frag',
                                      var_y='Dupl_rate',
                                      min_x=QC_filters['Log_unique_nr_frag'][0],
                                      max_x=QC_filters['Log_unique_nr_frag'][1],
                                      min_y=QC_filters['Dupl_rate'][0],
                                      max_y=QC_filters['Dupl_rate'][1],
                                      return_cells=False,
                                      return_fig=True,
                                      plot=False,
                                      plot_as_hexbin = True)

# Plot barcode stats in one figure
fig=plt.figure(figsize=(10,10))
plt.subplot(1, 3, 1)
img = fig2img(FRIP_NR_FRAG_fig)
plt.imshow(img)
plt.axis('off')
plt.subplot(1, 3, 2)
img = fig2img(TSS_NR_FRAG_fig)
plt.imshow(img)
plt.axis('off')
plt.subplot(1, 3, 3)
img = fig2img(DR_NR_FRAG_fig)
plt.imshow(img)
plt.axis('off')
plt.show()

In [ ]:
bc_passing_filters = {'E4AD_1yr':[]}
bc_passing_filters['E4AD_1yr'] = list((set(FRIP_NR_FRAG_filter) & set(TSS_NR_FRAG_filter)))
pickle.dump(bc_passing_filters,
            open('E4AD_1yr/quality_control/bc_passing_filters.pkl', 'wb'))
print(f"{len(bc_passing_filters['E4AD_1yr'])} barcodes passed QC stats")

### Topic Modeling

In [ ]:
import scanpy as sc
adata = sc.read_h5ad('E4AD_1yr/rna_adata.h5ad')
scRNA_bc = adata.obs_names
cell_data = adata.obs
cell_data['sample_id'] = 'E4AD_1yr'
cell_data['celltype'] = cell_data['celltype'].astype(str) # set data type of the celltype column to str, otherwise the export_pseudobulk function will complain.
del(adata)

In [ ]:
import pickle
fragments_dict = {'E4AD_1yr': '/lustre/fs4/home/amillet/scenicplus/E4AD_1yr/atac_fragments_filtered.tsv.gz'}
path_to_regions = {'E4AD_1yr': 'E4AD_1yr/consensus_peak_calling/consensus_regions.bed'}
path_to_blacklist= 'mm10-blacklist.v2.bed'
metadata_bc = pickle.load(open('E4AD_1yr/quality_control/metadata_bc.pkl', 'rb'))
bc_passing_filters = pickle.load(open('E4AD_1yr/quality_control/bc_passing_filters.pkl', 'rb'))

In [ ]:
print(f"{len(list(set(bc_passing_filters['E4AD_1yr']) & set(scRNA_bc)))} cell barcodes pass both scATAC-seq and scRNA-seq based filtering")


In [ ]:
from pycisTopic.cistopic_class import *
key = 'E4AD_1yr'
cistopic_obj = create_cistopic_object_from_fragments(
                            path_to_fragments=fragments_dict[key],
                            path_to_regions=path_to_regions[key],
                            path_to_blacklist=path_to_blacklist,
                            metrics=metadata_bc[key],
                            valid_bc=list(set(scRNA_bc)), # removing `set(bc_passing_filters[key]) & ` as RNA is already prefiltered on both 
                            n_cpu=1,
                            project=key,
                            split_pattern='-')
cistopic_obj.add_cell_data(cell_data, split_pattern='-')
print(cistopic_obj)

In [ ]:
pickle.dump(cistopic_obj,
            open('E4AD_1yr/cistopic_obj.pkl', 'wb'))

In [ ]:
# import pickle
cistopic_obj = pickle.load(open('E4AD_1yr/cistopic_obj.pkl', 'rb'))
from pycisTopic.cistopic_class import *
models=run_cgs_models(cistopic_obj,
                    n_topics=[2,4,10,16,32,48],
                    n_cpu=5,
                    n_iter=500,
                    random_state=555,
                    alpha=50,
                    alpha_by_topic=True,
                    eta=0.1,
                    eta_by_topic=False,
                    save_path=None,
                    _temp_dir = "/lustre/fs4/home/amillet/ray_spill")

In [ ]:
if not os.path.exists('E4AD_1yr/models'):
    os.makedirs('E4AD_1yr/models')

pickle.dump(models,
            open('E4AD_1yr/models/10x_pbmc_models_500_iter_LDA.pkl', 'wb'))

In [ ]:
models = pickle.load(open('E4AD_1yr/models/10x_pbmc_models_500_iter_LDA.pkl', 'rb'))
cistopic_obj = pickle.load(open('E4AD_1yr/cistopic_obj.pkl', 'rb'))
from pycisTopic.lda_models import *
model = evaluate_models(models,
                       select_model=32,
                       return_model=True,
                       metrics=['Arun_2010','Cao_Juan_2009', 'Minmo_2011', 'loglikelihood'],
                       plot_metrics=False)

In [ ]:
cistopic_obj.add_LDA_model(model)
pickle.dump(cistopic_obj,
            open('E4AD_1yr/cistopic_obj.pkl', 'wb'))

In [ ]:
from pycisTopic.clust_vis import *
run_umap(cistopic_obj, target  = 'cell', scale=True)
plot_metadata(cistopic_obj, reduction_name = 'UMAP', variables = ['celltype'])

In [ ]:
plot_topic(cistopic_obj, reduction_name = 'UMAP', num_columns = 4)

In [ ]:
from pycisTopic.topic_binarization import *
region_bin_topics_otsu = binarize_topics(cistopic_obj, method='otsu')
region_bin_topics_top3k = binarize_topics(cistopic_obj, method='ntop', ntop = 3000)

In [ ]:
import numpy as np
from pycisTopic.diff_features import *
imputed_acc_obj = impute_accessibility(cistopic_obj, selected_cells=None, selected_regions=None, scale_factor=10**6)
normalized_imputed_acc_obj = normalize_scores(imputed_acc_obj, scale_factor=10**4)
variable_regions = find_highly_variable_features(normalized_imputed_acc_obj, plot = False)
markers_dict = find_diff_features(cistopic_obj, imputed_acc_obj, variable='celltype', 
                                  var_features=variable_regions, split_pattern = '-',
                                 adjpval_thr = 0.1, log2fc_thr = np.log2(1.1))

In [ ]:
if not os.path.exists('E4AD_1yr/candidate_enhancers'):
    os.makedirs('E4AD_1yr/candidate_enhancers')
import pickle
pickle.dump(region_bin_topics_otsu, open('E4AD_1yr/candidate_enhancers/region_bin_topics_otsu.pkl', 'wb'))
pickle.dump(region_bin_topics_top3k, open('E4AD_1yr/candidate_enhancers/region_bin_topics_top3k.pkl', 'wb'))
pickle.dump(markers_dict, open('E4AD_1yr/candidate_enhancers/markers_dict.pkl', 'wb'))

## PyCisTarget Motif Enrichment Analysis

In [ ]:
import pickle
region_bin_topics_otsu = pickle.load(open('E4AD_1yr/candidate_enhancers/region_bin_topics_otsu.pkl', 'rb'))
region_bin_topics_top3k = pickle.load(open('E4AD_1yr/candidate_enhancers/region_bin_topics_top3k.pkl', 'rb'))
markers_dict = pickle.load(open('E4AD_1yr/candidate_enhancers/markers_dict.pkl', 'rb'))

In [ ]:
import pyranges as pr
from pycistarget.utils import region_names_to_coordinates
region_sets = {}
region_sets['topics_otsu'] = {}
region_sets['topics_top_3'] = {}
region_sets['DARs'] = {}
for topic in region_bin_topics_otsu.keys():
    regions = region_bin_topics_otsu[topic].index[region_bin_topics_otsu[topic].index.str.startswith('chr')] #only keep regions on known chromosomes
    region_sets['topics_otsu'][topic] = pr.PyRanges(region_names_to_coordinates(regions))
for topic in region_bin_topics_top3k.keys():
    regions = region_bin_topics_top3k[topic].index[region_bin_topics_top3k[topic].index.str.startswith('chr')] #only keep regions on known chromosomes
    region_sets['topics_top_3'][topic] = pr.PyRanges(region_names_to_coordinates(regions))
for DAR in markers_dict.keys():
    regions = markers_dict[DAR].index[markers_dict[DAR].index.str.startswith('chr')] #only keep regions on known chromosomes
    region_sets['DARs'][DAR] = pr.PyRanges(region_names_to_coordinates(regions))

In [ ]:
for key in region_sets.keys():
    print(f'{key}: {region_sets[key].keys()}')

In [ ]:
rankings_db = '/ru-auth/local/home/amillet/scratch/references/cistarget/mm10_screen_v10_clust.regions_vs_motifs.rankings.feather'
scores_db = '/ru-auth/local/home/amillet/scratch/references/cistarget/mm10_screen_v10_clust.regions_vs_motifs.scores.feather'
motif_annotation = '/ru-auth/local/home/amillet/scratch/references/cistarget/motifs-v10nr_clust-nr.mgi-m0.001-o0.0.tbl'

In [ ]:
if not os.path.exists('E4AD_1yr/motifs'):
    os.makedirs('E4AD_1yr/motifs')
from scenicplus.wrappers.run_pycistarget import run_pycistarget
run_pycistarget(
    region_sets = region_sets,
    species = 'mus_musculus',
    save_path = 'E4AD_1yr/motifs',
    ctx_db_path = rankings_db,
    dem_db_path = scores_db,
    path_to_motif_annotations = motif_annotation,
    run_without_promoters = True,
    n_cpu = 8,
    _temp_dir = "/lustre/fs4/home/amillet/ray_spill",
    annotation_version = 'v10nr_clust',
    )

In [ ]:
import dill
menr = dill.load(open('E4AD_1yr/motifs/menr.pkl', 'rb'))

In [ ]:
menr['DEM_topics_otsu_All'].DEM_results('Topic17')

# Finally!! Time to run SCENIC+.

In [ ]:
import dill
import scanpy as sc
import os
import warnings
warnings.filterwarnings("ignore")
import pandas
import pyranges
# Set stderr to null to avoid strange messages from ray
import sys
_stderr = sys.stderr
null = open(os.devnull,'wb')

adata = sc.read_h5ad('E4AD_1yr/rna_adata.h5ad')
cistopic_obj = dill.load(open('E4AD_1yr/cistopic_obj.pkl', 'rb'))
menr = dill.load(open('E4AD_1yr/motifs/menr.pkl', 'rb'))

In [ ]:
from scenicplus.scenicplus_class import create_SCENICPLUS_object
import numpy as np
scplus_obj = create_SCENICPLUS_object(
    GEX_anndata = adata.raw.to_adata(),
    cisTopic_obj = cistopic_obj,
    menr = menr,
    bc_transform_func = lambda x: f'{x}-E4AD_1yr' #function to convert scATAC-seq barcodes to scRNA-seq ones
)
scplus_obj.X_EXP = np.array(scplus_obj.X_EXP.todense())
scplus_obj

Check which biomart host is best:

In [ ]:
ensembl_version_dict = {'105': 'http://www.ensembl.org',
                        '104': 'http://may2021.archive.ensembl.org/',
                        '103': 'http://feb2021.archive.ensembl.org/',
                        '102': 'http://nov2020.archive.ensembl.org/',
                        '101': 'http://aug2020.archive.ensembl.org/',
                        '100': 'http://apr2020.archive.ensembl.org/',
                        '99': 'http://jan2020.archive.ensembl.org/',
                        '98': 'http://sep2019.archive.ensembl.org/',
                        '97': 'http://jul2019.archive.ensembl.org/',
                        '96': 'http://apr2019.archive.ensembl.org/',
                        '95': 'http://jan2019.archive.ensembl.org/',
                        '94': 'http://oct2018.archive.ensembl.org/',
                        '93': 'http://jul2018.archive.ensembl.org/',
                        '92': 'http://apr2018.archive.ensembl.org/',
                        '91': 'http://dec2017.archive.ensembl.org/',
                        '90': 'http://aug2017.archive.ensembl.org/',
                        '89': 'http://may2017.archive.ensembl.org/',
                        '88': 'http://mar2017.archive.ensembl.org/',
                        '87': 'http://dec2016.archive.ensembl.org/',
                        '86': 'http://oct2016.archive.ensembl.org/',
                        '80': 'http://may2015.archive.ensembl.org/',
                        '77': 'http://oct2014.archive.ensembl.org/',
                        '75': 'http://feb2014.archive.ensembl.org/',
                        '54': 'http://may2009.archive.ensembl.org/'}

import pybiomart as pbm
def test_ensembl_host(scplus_obj, host, species):
    dataset = pbm.Dataset(name=species+'_gene_ensembl',  host=host)
    annot = dataset.query(attributes=['chromosome_name', 'transcription_start_site', 'strand', 'external_gene_name', 'transcript_biotype'])
    annot.columns = ['Chromosome', 'Start', 'Strand', 'Gene', 'Transcript_type']
    annot['Chromosome'] = annot['Chromosome'].astype('str')
    filter = annot['Chromosome'].str.contains('CHR|GL|JH|MT')
    annot = annot[~filter]
    annot.columns=['Chromosome', 'Start', 'Strand', 'Gene', 'Transcript_type']
    gene_names_release = set(annot['Gene'].tolist())
    ov=len([x for x in scplus_obj.gene_names if x in gene_names_release])
    print('Genes recovered: ' + str(ov) + ' out of ' + str(len(scplus_obj.gene_names)))
    return ov

n_overlap = {}
for version in ensembl_version_dict.keys():
    print(f'host: {version}')
    try:
        n_overlap[version] =  test_ensembl_host(scplus_obj, ensembl_version_dict[version], 'mmusculus')
    except:
        print('Host not reachable')
v = sorted(n_overlap.items(), key=lambda item: item[1], reverse=True)[0][0]
print(f"version: {v} has the largest overlap, use {ensembl_version_dict[v]} as biomart host")

In [ ]:
biomart_host = "http://sep2019.archive.ensembl.org/"

We prep a list of all known mouse TFs, first by downloading the annotated txt file from http://bioinfo.life.hust.edu.cn/AnimalTFDB4/#/Download for mouse and saving it to `scenicplus/Mus_musculus_TF.txt`. Then we filter to just the TF names and save as a new txt file for use.

In [ ]:
import pandas as pd
import numpy as np
df = pd.read_csv("Mus_musculus_TF.txt", sep = "\t")
df = df['Symbol']
np.savetxt(r'Mus_musculus_TF_readytouse.txt', df.values, fmt='%s')

In [ ]:
scplus_obj.dr_cell['GEX_X_pca'] = scplus_obj.dr_cell['GEX_X_pca'].iloc[:, 0:2]

In [ ]:
from scenicplus.wrappers.run_scenicplus import run_scenicplus
try:
    run_scenicplus(
        scplus_obj = scplus_obj,
        variable = ['GEX_celltype'],
        species = 'mmusculus',
        assembly = 'mm10',
        tf_file = 'Mus_musculus_TF_readytouse.txt',
        save_path = 'E4AD_1yr/scenicplus',
        biomart_host = biomart_host,
        upstream = [1000, 150000],
        downstream = [1000, 150000],
        calculate_TF_eGRN_correlation = True,
        calculate_DEGs_DARs = True,
        export_to_loom_file = True,
        export_to_UCSC_file = True,
        path_bedToBigBed = '/ru-auth/local/home/amillet/scenicplus',
        n_cpu = 12,
        _temp_dir = "/lustre/fs4/home/amillet/ray_spill")
except Exception as e:
    #in case of failure, still save the object
    dill.dump(scplus_obj, open('E4AD_1yr/scenicplus/scplus_obj.pkl', 'wb'), protocol=-1)
    raise(e)

Note: for this code to run, had to edit `src/scenicplus/loom.py` line 174 from `), columns=cv.get_feature_names(), index=regulons.keys())` to `), columns=cv.get_feature_names_out(), index=regulons.keys())` due to an update in scikit-learn. See: https://github.com/aertslab/scenicplus/issues/76

In [ ]:
scplus_obj

## FINALLY time to explore the data!

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import sys
import os
_stderr = sys.stderr
null = open(os.devnull,'wb')

In [ ]:
import dill
scplus_obj = dill.load(open('E4AD_1yr/scenicplus/scplus_obj.pkl', 'rb'))

In [ ]:
from scenicplus.preprocessing.filtering import apply_std_filtering_to_eRegulons
apply_std_filtering_to_eRegulons(scplus_obj)

In [ ]:
work_dir = "E4AD_1yr"
from scenicplus.eregulon_enrichment import score_eRegulons
region_ranking = dill.load(open(os.path.join(work_dir, 'scenicplus/region_ranking.pkl'), 'rb')) #load ranking calculated using the wrapper function
gene_ranking = dill.load(open(os.path.join(work_dir, 'scenicplus/gene_ranking.pkl'), 'rb')) #load ranking calculated using the wrapper function
score_eRegulons(scplus_obj,
                ranking = region_ranking,
                eRegulon_signatures_key = 'eRegulon_signatures_filtered',
                key_added = 'eRegulon_AUC_filtered',
                enrichment_type= 'region',
                auc_threshold = 0.05,
                normalize = False,
                n_cpu = 5)
score_eRegulons(scplus_obj,
                gene_ranking,
                eRegulon_signatures_key = 'eRegulon_signatures_filtered',
                key_added = 'eRegulon_AUC_filtered',
                enrichment_type = 'gene',
                auc_threshold = 0.05,
                normalize= False,
                n_cpu = 5)

In [ ]:
from scenicplus.dimensionality_reduction import run_eRegulons_tsne, run_eRegulons_umap
run_eRegulons_umap(
    scplus_obj = scplus_obj,
    auc_key = 'eRegulon_AUC_filtered',
    reduction_name = 'eRegulons_UMAP', #overwrite previously calculated UMAP
)
run_eRegulons_tsne(
    scplus_obj = scplus_obj,
    auc_key = 'eRegulon_AUC_filtered',
    reduction_name = 'eRegulons_tSNE', #overwrite previously calculated tSNE
)

In [ ]:
scplus_obj.metadata_cell['GEX_celltype'].value_counts()

In [ ]:
from scenicplus.dimensionality_reduction import plot_metadata_given_ax
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

#specify color_dictionary

color_dict = {
    'Homeostatic Microglia': "#065143",
    'Selplg-lo Microglia': "#70B77E",
    'mt-Enriched Microglia': "#E0A890",
    'mt-Depleted Microglia': "#053C5E",
    'DAM-1': "#F56476",
    'DAM-2': "#CE1483" ,
    'TIMs': "#38A3A5",
    'Siglech-hi Microglia': "#80ED99"
}

fig, axs = plt.subplots(ncols=2, figsize = (16, 8))
plot_metadata_given_ax(
    scplus_obj=scplus_obj,
    ax = axs[0],
    reduction_name = 'eRegulons_UMAP',
    variable = 'GEX_celltype', #note the GEX_ prefix, this metadata originated from the gene expression metadata (on which we did the cell type annotation before)
    color_dictionary={'GEX_celltype': color_dict}
)
plot_metadata_given_ax(
    scplus_obj=scplus_obj,
    ax = axs[1],
    reduction_name = 'eRegulons_tSNE',
    variable = 'GEX_celltype', #note the GEX_ prefix, this metadata originated from the gene expression metadata (on which we did the cell type annotation before)
    color_dictionary={'GEX_celltype': color_dict}
)
fig.tight_layout()
sns.despine(ax = axs[0]) #remove top and right edge of axis border
sns.despine(ax = axs[1]) #remove top and right edge of axis border
plt.show()

In [ ]:
from scenicplus.dimensionality_reduction import plot_eRegulon
plot_eRegulon(
    scplus_obj = scplus_obj,
    reduction_name = 'eRegulons_UMAP',
    selected_regulons = ['Fos_+', 'Jun_+', 'Fosb_+', 'Klf4_+', 'Egr3_+'],
    scale = True,
    auc_key = 'eRegulon_AUC_filtered')

In [ ]:
df = scplus_obj.uns['eRegulon_metadata_filtered']
df[df.TF == "Jun"]

In [ ]:
from scenicplus.dimensionality_reduction import plot_AUC_given_ax

fig, ax = plt.subplots(figsize = (8,8))
plot_AUC_given_ax(
    scplus_obj = scplus_obj,
    reduction_name = 'eRegulons_tSNE',
    feature = 'Jun_+_(73g)',
    ax = ax,
    auc_key = 'eRegulon_AUC_filtered',
    signature_key = 'Gene_based')
sns.despine(ax = ax)
plt.show()

In [ ]:
from scenicplus.cistromes import TF_cistrome_correlation, generate_pseudobulks

generate_pseudobulks(
        scplus_obj = scplus_obj,
        variable = 'GEX_celltype',
        auc_key = 'eRegulon_AUC_filtered',
        signature_key = 'Gene_based')
generate_pseudobulks(
        scplus_obj = scplus_obj,
        variable = 'GEX_celltype',
        auc_key = 'eRegulon_AUC_filtered',
        signature_key = 'Region_based')

TF_cistrome_correlation(
            scplus_obj,
            use_pseudobulk = True,
            variable = 'GEX_celltype',
            auc_key = 'eRegulon_AUC_filtered',
            signature_key = 'Gene_based',
            out_key = 'filtered_gene_based')
TF_cistrome_correlation(
            scplus_obj,
            use_pseudobulk = True,
            variable = 'GEX_celltype',
            auc_key = 'eRegulon_AUC_filtered',
            signature_key = 'Region_based',
            out_key = 'filtered_region_based')

In [ ]:
scplus_obj.uns['TF_cistrome_correlation']['filtered_region_based'].head()

In [ ]:
import numpy as np
n_targets = [int(x.split('(')[1].replace('r)', '')) for x in scplus_obj.uns['TF_cistrome_correlation']['filtered_region_based']['Cistrome']]
rho = scplus_obj.uns['TF_cistrome_correlation']['filtered_region_based']['Rho'].to_list()
adj_pval = scplus_obj.uns['TF_cistrome_correlation']['filtered_region_based']['Adjusted_p-value'].to_list()

thresholds = {
        'rho': [-0.15, 0.15],
        'n_targets': 0
}
import seaborn as sns
fig, ax = plt.subplots(figsize = (10, 5))
sc = ax.scatter(rho, n_targets, c = -np.log10(adj_pval), s = 5)
ax.set_xlabel('Correlation coefficient')
ax.set_ylabel('nr. target regions')
#ax.hlines(y = thresholds['n_targets'], xmin = min(rho), xmax = max(rho), color = 'black', ls = 'dashed', lw = 1)
ax.vlines(x = thresholds['rho'], ymin = 0, ymax = max(n_targets), color = 'black', ls = 'dashed', lw = 1)
ax.text(x = thresholds['rho'][0], y = max(n_targets), s = str(thresholds['rho'][0]))
ax.text(x = thresholds['rho'][1], y = max(n_targets), s = str(thresholds['rho'][1]))
sns.despine(ax = ax)
fig.colorbar(sc, label = '-log10(adjusted_pvalue)', ax = ax)
plt.show()

In [ ]:
selected_cistromes = scplus_obj.uns['TF_cistrome_correlation']['filtered_region_based'].loc[
        np.logical_or(
                scplus_obj.uns['TF_cistrome_correlation']['filtered_region_based']['Rho'] > thresholds['rho'][1],
                scplus_obj.uns['TF_cistrome_correlation']['filtered_region_based']['Rho'] < thresholds['rho'][0]
        )]['Cistrome'].to_list()
selected_eRegulons = [x.split('_(')[0] for x in selected_cistromes]
selected_eRegulons_gene_sig = [
        x for x in scplus_obj.uns['eRegulon_signatures_filtered']['Gene_based'].keys()
        if x.split('_(')[0] in selected_eRegulons]
selected_eRegulons_region_sig = [
        x for x in scplus_obj.uns['eRegulon_signatures_filtered']['Region_based'].keys()
        if x.split('_(')[0] in selected_eRegulons]
#save the results in the scenicplus object
scplus_obj.uns['selected_eRegulon'] = {'Gene_based': selected_eRegulons_gene_sig, 'Region_based': selected_eRegulons_region_sig}
print(f'selected: {len(selected_eRegulons_gene_sig)} eRegulons')

In [ ]:
dill.dump(scplus_obj, open(os.path.join(work_dir, 'scenicplus/scplus_obj.pkl'), 'wb'), protocol=-1)

In [ ]:
from scenicplus.plotting.dotplot import heatmap_dotplot
heatmap_dotplot(
        scplus_obj = scplus_obj,
        size_matrix = scplus_obj.uns['eRegulon_AUC_filtered']['Region_based'], #specify what to plot as dot sizes, target region enrichment in this case
        color_matrix = scplus_obj.to_df('EXP'), #specify  what to plot as colors, TF expression in this case
        scale_size_matrix = True,
        scale_color_matrix = True,
        group_variable = 'GEX_celltype',
        subset_eRegulons = scplus_obj.uns['selected_eRegulon']['Gene_based'],
        index_order = ['Homeostatic Microglia', 'Selplg-lo Microglia', 'mt-Enriched Microglia', 'mt-Depleted Microglia', 'DAM-1', 'DAM-2', 'TIMs', 'Siglech-hi Microglia'],
        figsize = (15, 20),
        orientation = 'vertical',
        split_repressor_activator = False,
        save = "eregulon_dotplot.png") # set to False since we get no repressors :hehe:

In [ ]:
from scenicplus.plotting.dotplot import generate_dotplot_df
df = generate_dotplot_df(scplus_obj = scplus_obj,
        size_matrix = scplus_obj.uns['eRegulon_AUC_filtered']['Region_based'], #specify what to plot as dot sizes, target region enrichment in this case
        color_matrix = scplus_obj.to_df('EXP'), #specify  what to plot as colors, TF expression in this case
        scale_size_matrix = True,
        scale_color_matrix = True,
        group_variable = 'GEX_celltype',
        subset_eRegulons = scplus_obj.uns['selected_eRegulon']['Gene_based'],)

In [ ]:
index_order = ['Homeostatic Microglia', 'Selplg-lo Microglia', 'mt-Enriched Microglia', 'mt-Depleted Microglia', 'DAM-1', 'DAM-2', 'TIMs', 'Siglech-hi Microglia']
tmp = df[['index', 'eRegulon_name', 'color_val']
        ].pivot_table(index = 'index', columns = 'eRegulon_name'
        ).fillna(0)['color_val']
tmp = tmp.loc[index_order]
idx_max = tmp.idxmax(axis = 0)
order = pd.concat([idx_max[idx_max == x] for x in tmp.index.tolist() if len(df[df == x]) > 0]).index.tolist()

In [ ]:
df.to_csv("eregulons_df.csv")
pd.DataFrame(order).to_csv("regulon_order.csv")

In [ ]:
from scenicplus.RSS import *
regulon_specificity_scores(
        scplus_obj,
        variable = 'GEX_celltype',
        auc_key = 'eRegulon_AUC_filtered',
        signature_keys = ['Region_based'],
        selected_regulons = [x for x in scplus_obj.uns['selected_eRegulon']['Region_based'] if '-' not in x],
        out_key_suffix = '_filtered')

In [ ]:
plot_rss(scplus_obj, 'GEX_celltype_filtered', num_columns=2, top_n=10, figsize = (10, 20))

In [ ]:
flat_list = lambda t: [item for sublist in t for item in sublist]
selected_markers = list(set(flat_list(
    [scplus_obj.uns['RSS']['GEX_celltype_filtered'].loc[celltype].sort_values(ascending = False).head(10).index.to_list()
    for celltype in scplus_obj.uns['RSS']['GEX_celltype_filtered'].index])))

In [ ]:
from scenicplus.plotting.correlation_plot import *

region_intersetc_data, Z = jaccard_heatmap(
        scplus_obj,
        method = 'intersect',
        gene_or_region_based = 'Region_based',
        use_plotly = False,
        selected_regulons = selected_markers,
        signature_key = 'eRegulon_signatures_filtered',
        figsize = (10, 10), return_data = True, vmax = 0.5, cmap = 'plasma')

## Perturbation simulation

In [ ]:
from scenicplus.dimensionality_reduction import run_eRegulons_pca
run_eRegulons_pca(
        scplus_obj,
        auc_key = 'eRegulon_AUC_filtered',
        reduction_name = 'eRegulons_PCA_gene_based',
        selected_regulons = scplus_obj.uns['selected_eRegulon']['Gene_based'])

In [ ]:
from pycisTopic.diff_features import find_highly_variable_features
hvg = find_highly_variable_features(scplus_obj.to_df('EXP')[list(set(scplus_obj.uns['eRegulon_metadata_filtered']['Gene']))].T, n_top_features = 200, plot = False)

In [ ]:
color_dict_line = {
    'Homeostatic Microglia': "#065143",
    'Selplg-lo Microglia': "#70B77E",
    'mt-Enriched Microglia': "#E0A890",
    'mt-Depleted Microglia': "#053C5E",
    'DAM-1': "#F56476",
    'DAM-2': "#CE1483" ,
    'TIMs': "#38A3A5",
    'Siglech-hi Microglia': "#80ED99"}

from scenicplus.simulation import plot_perturbation_effect_in_embedding
import seaborn as sns
_ = plot_perturbation_effect_in_embedding(
        scplus_obj = scplus_obj,
        reduction_name = 'eRegulons_PCA_gene_based',
        n_cpu = 5,
        perturbation = {'Fos': 0}, #specifies that we want to set the expression of Fos to 0 in all cells.
        variable = 'GEX_celltype',
        color_dictionary = {'GEX_celltype': color_dict_line},
        genes_to_use = hvg,
        figsize = (5, 5))

In [ ]:
color_dict_line = {
    'Homeostatic Microglia': "#065143",
    'Selplg-lo Microglia': "#70B77E",
    'mt-Enriched Microglia': "#E0A890",
    'mt-Depleted Microglia': "#053C5E",
    'DAM-1': "#F56476",
    'DAM-2': "#CE1483" ,
    'TIMs': "#38A3A5",
    'Siglech-hi Microglia': "#80ED99"}

from scenicplus.simulation import plot_perturbation_effect_in_embedding
import seaborn as sns
_ = plot_perturbation_effect_in_embedding(
        scplus_obj = scplus_obj,
        reduction_name = 'eRegulons_PCA_gene_based',
        n_cpu = 5,
        perturbation = {'Sox5': 0}, #specifies that we want to set the expression of Sox5 to 0 in all cells.
        variable = 'GEX_celltype',
        color_dictionary = {'GEX_celltype': color_dict_line},
        genes_to_use = hvg,
        figsize = (5, 5))